In [1]:
import numpy as np
from functools import reduce

In [2]:
zero = np.array([1, 0])
one = np.array([0, 1])

In [3]:
zero2 = np.kron(zero, zero)

In [4]:
zero2

array([1, 0, 0, 0])

In [5]:
np.outer(zero, zero)

array([[1, 0],
       [0, 0]])

In [6]:
def zeron(n):
    '''
    Generate |00000...> state
    '''
    zero = np.array([1, 0])
    return reduce(np.kron, [zero]*n)

In [7]:
np.outer(zeron(2), zeron(2))

array([[1, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

In [8]:
def bitState(bString):
    '''
    Generate bit string state
    
    Args
    ----
    bString : String of 0s and 1s describing state
    '''
    stateDict = {
        '0' : np.array([1, 0]),
        '1' : np.array([0, 1])
    }

    bArr = [stateDict[b] for b in bString]
    return reduce(np.kron, bArr)

def bitProjector(bString):
    '''
    Generate projector for given bitString state
    '''
    state = bitState(bString)
    return np.outer(state, state)

In [9]:
bitState('10')

array([0, 0, 1, 0])

In [10]:
np.kron(zero, one)

array([0, 1, 0, 0])

In [11]:
bitProjector('10')

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 0]])

Generate random 4 qubit state. Trace out the first two qubits and project on the last two. 

In [12]:
nqubits = 4
dim = 2**4
phi = np.random.rand(dim) + 1j*np.random.rand(dim) # Generate a random state
norm = np.linalg.norm(phi)
phi = phi / norm # normalise state

In [13]:
phi @ np.conj(phi)

(1.0000000000000002+0j)

In [14]:
import qiskit
import qiskit.quantum_info as qi

In [15]:
state = qi.Statevector(bitState('10'))

In [16]:
state.draw(output='latex')

<IPython.core.display.Latex object>

In [17]:
dMatrix = qi.DensityMatrix(state)

In [18]:
dMatrix

DensityMatrix([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
               [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
               [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
               [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]],
              dims=(2, 2))


In [19]:
qi.partial_trace(dMatrix, [0])

DensityMatrix([[0.+0.j, 0.+0.j],
               [0.+0.j, 1.+0.j]],
              dims=(2,))


In [20]:
qi.DensityMatrix(bitState('1'))

DensityMatrix([[0.+0.j, 0.+0.j],
               [0.+0.j, 1.+0.j]],
              dims=(2,))


In [21]:
def partialTrace(state, tracedInds):
    
    nqubits = int(np.log2(state.shape[0]))
    state = state.reshape([2]*nqubits)
    
    traced = np.tensordot(state, state.conj(), [tracedInds, tracedInds])

    nRemaining = len(traced.shape) // 2
    return traced.reshape([2**nRemaining]*2)

In [22]:
partialTrace(bitState('10'), [0])

array([[1, 0],
       [0, 0]])

Verifying the behaviour of the projector

In [23]:
state = np.random.randn(4) + 1j*np.random.rand(4)
state = state / np.linalg.norm(state)

In [24]:
phi = np.outer(state, state)

In [25]:
phi

array([[ 0.14806355-0.23571412j, -0.06187917+0.01995396j,
         0.28973013-0.29702845j, -0.14885483-0.04817723j],
       [-0.06187917+0.01995396j,  0.0140684 +0.00571817j,
        -0.09643397+0.00965996j,  0.01940502+0.03096617j],
       [ 0.28973013-0.29702845j, -0.09643397+0.00965996j,
         0.51541189-0.34192294j, -0.20179837-0.11691602j],
       [-0.14885483-0.04817723j,  0.01940502+0.03096617j,
        -0.20179837-0.11691602j, -0.00572651+0.08775292j]])

In [26]:
proj = bitProjector('00')

In [27]:
proj

array([[1, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

In [28]:
np.trace(proj @ phi)

(0.1480635466057828-0.2357141193358807j)

In [29]:
proj @ phi

array([[ 0.14806355-0.23571412j, -0.06187917+0.01995396j,
         0.28973013-0.29702845j, -0.14885483-0.04817723j],
       [ 0.        +0.j        ,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ],
       [ 0.        +0.j        ,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ],
       [ 0.        +0.j        ,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ]])

In [30]:
bin(4)[2:].zfill(5)

'00100'

In [31]:
bstrings = [bin(i)[2:].zfill(4) for i in range(10)]

for b in bstrings:
    print(b)
    print(np.argwhere(bitProjector(b)))

0000
[[0 0]]
0001
[[1 1]]
0010
[[2 2]]
0011
[[3 3]]
0100
[[4 4]]
0101
[[5 5]]
0110
[[6 6]]
0111
[[7 7]]
1000
[[8 8]]
1001
[[9 9]]


Verify efficient calculation of projection

In [32]:
nQb = 4
phi = np.random.randn(2**nQb) + 1j*np.random.randn(2**nQb) 
phi = phi / np.linalg.norm(phi)

rho = np.outer(phi.conj(), phi)

In [33]:
def projectDensity(dMatrix, bString):
    #dim = dMatrix.shape[0]
    #assert 2**len(bString) == dim
    i = int(bString, 2)
    return dMatrix[i, i]

In [34]:
rho.shape

(16, 16)

In [35]:
bStrings = ['0000', '0001', '0101', '1000']
for b in bStrings:
    print(b)
    projector = bitProjector(b)
    print(projector.shape)
    projDirect = np.trace(bitProjector(b) @ rho)
    projLookup = projectDensity(rho, b)
    print(np.allclose(projDirect, projLookup))
    print()

0000
(16, 16)
True

0001
(16, 16)
True

0101
(16, 16)
True

1000
(16, 16)
True



In [36]:
bStrings = ['0000', '0001', '0101', '1000']
projs = np.zeros(len(bStrings))

In [37]:
%%timeit
for i, b in enumerate(bStrings):
    projector = bitProjector(b)
    projDirect = np.real(np.trace(bitProjector(b) @ rho))
    projs[i] = projDirect

616 µs ± 53.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [38]:
bStrings = ['0000', '0001', '0101', '1000']
projs = np.zeros(len(bStrings))

In [39]:
%%timeit
for i, b in enumerate(bStrings):
    projLookup = projectDensity(rho, b)
    projs[i] = np.real(projLookup)

4.29 µs ± 171 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [40]:
bStrings = ['0000', '0001', '0101', '1000']
projs = np.zeros(len(bStrings))

In [41]:
%%timeit
for i, b in enumerate(bStrings):
    projLookup = projectDensity(rho, b)
    projs[i] = np.real(projLookup)

4.27 µs ± 186 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Now finding an efficent way to calculate $dOdV$

In [42]:
nQb = 4
state = np.arange(2**nQb)

In [43]:
state

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [44]:
bStrings = ['0000', '0001', '0101', '1000']

for b in bStrings:
    print(b)
    print(int(b, 2))
    
    proj = bitProjector(b) @ state
    print(proj)
    print(np.argwhere(proj))

0000
0
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[]
0001
1
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[[1]]
0101
5
[0 0 0 0 0 5 0 0 0 0 0 0 0 0 0 0]
[[5]]
1000
8
[0 0 0 0 0 0 0 0 8 0 0 0 0 0 0 0]
[[8]]


In [45]:
U = np.arange(2**(nQb*2)).reshape(2**nQb, -1)

In [46]:
U

array([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15],
       [ 16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,
         29,  30,  31],
       [ 32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,
         45,  46,  47],
       [ 48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,
         61,  62,  63],
       [ 64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,
         77,  78,  79],
       [ 80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
         93,  94,  95],
       [ 96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108,
        109, 110, 111],
       [112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124,
        125, 126, 127],
       [128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140,
        141, 142, 143],
       [144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
        157, 158, 159],
       [160, 161, 162, 163, 16

In [47]:
bitProjector('0000') @ U

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,

In [48]:
bitProjector('0000') @ U @ state

array([1240,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0])

In [49]:
def PV_state(bitString, V, phi):
    i = int(bitString, 2)
    Vrow = V[i, :]
    out = np.zeros(phi.shape)
    out[i] = np.dot(Vrow, phi)
    return out

In [50]:
PV_state('0000', U, state)

array([1240.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.])

In [51]:
state

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [52]:
state_bit = np.zeros(state.shape)
state_bit[0] = 2

In [53]:
state_bit

array([2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [54]:
np.outer(state_bit, state)

array([[ 0.,  2.,  4.,  6.,  8., 10., 12., 14., 16., 18., 20., 22., 24.,
        26., 28., 30.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0

Putting this together with the outer produce we can efficiently calculate dOdV

In [55]:
def dOdV_efficient(bitString, V, state):
    i = int(bitString, 2)
    s = np.dot(V[i, :], state)
    out = np.zeros((state.shape[0], state.shape[0]), dtype=complex)
    out[i, :] = s*state.conj()
    return out

In [56]:
def dOdV_direct(bitString, V, state):
    P = bitProjector(bitString)
    PVstate = P @ V @ state
    return np.outer(PVstate, state.conj())

In [57]:
nQb = 4
U = np.random.randn(2**nQb, 2**nQb) + 1j*np.random.randn(2**nQb, 2**nQb)
state = np.random.randn(2**nQb) + 1j*np.random.randn(2**nQb)

In [58]:
bitStrings = ['0000', '0101', '0011', '1000', '1100']
for b in bitStrings:
    print(b)
    direct = dOdV_direct(b, U, state)
    efficient = dOdV_efficient(b, U, state)
    print(np.allclose(direct, efficient))

0000
True
0101
True
0011
True
1000
True
1100
True


In [59]:
%%timeit
for b in bitStrings:
    direct = dOdV_direct(b, U, state)

417 µs ± 11.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [60]:
%%timeit
for b in bitStrings:
    efficient = dOdV_efficient(b, U, state)

23.4 µs ± 657 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


$$\require{physics}$$

Checking if we can efficiently calculate $<\phi | V^\dagger P_i | V | \phi>$ using $s = V_i \cdot |\phi>$ and $s^*s$

In [61]:
nQb = 4
V = np.random.randn(2**nQb, 2**nQb) + 1j*np.random.randn(2**nQb, 2**nQb)
state = np.random.randn(2**nQb) + 1j*np.random.randn(2**nQb)

In [67]:
def overlapEfficient(bitString, V, state):
    i = int(bitString, 2)
    s = np.dot(V[i, :], state)
    return s.conj()*s

In [68]:
def overlapDirect(bitString, V, state):
    proj = bitProjector(b)
    return state.conj() @ V.conj().T @ proj @ V @ state

In [70]:
bitStrings = ['0000', '0010', '0100', '1011']
for b in bitStrings:
    oEff = overlapEfficient(b, V, state)
    oDir = overlapDirect(b, V, state)
    print(b)
    print(np.allclose(oEff, oDir))
    print()

0000
True

0010
True

0100
True

1011
True



In [71]:
%%timeit
for b in bitStrings:
    oDir = overlapDirect(b, V, state)

316 µs ± 17.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [73]:
%%timeit
for b in bitStrings:
    oEff = overlapEfficient(b, V, state)

9.53 µs ± 83.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Figuring out the effects of trace projections

In [97]:
nQb = 3
dims = [2]*(nQb)

state = np.arange(2**(nQb))

In [98]:
state = state.reshape(*dims)

In [99]:
state

array([[[0, 1],
        [2, 3]],

       [[4, 5],
        [6, 7]]])

In [100]:
state.shape

(2, 2, 2)

In [106]:
out = partialTrace(state.reshape(-1), [0, 1])

In [107]:
state.reshape(-1).shape

(8,)

In [108]:
out.shape

(2, 2)

In [109]:
out

array([[56, 68],
       [68, 84]])

In [111]:
np.dot(state[:, :, 0].reshape(-1), state[:, :, 0].reshape(-1))

56

In [113]:
np.dot(state[:, :, 0].reshape(-1), state[:, :, 1].reshape(-1))

68

In [96]:
np.dot(state[:, 1], state[:, 1])

10

In [93]:
np.dot(state[:, 1], state[:, 1])

10

In [114]:
np.allclose(state[:, :, 0], state.take(indices=0, axis=2))

True

In [116]:
np.allclose(state[slice(None), slice(None), 0], state.take(indices=0, axis=2))

True

In [117]:
%%timeit
state[slice(None), slice(None), 0]

308 ns ± 9.88 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [118]:
%%timeit
state.take(indices=0, axis=2)

472 ns ± 10.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [126]:
def tracedProjEfficient(state, ind, tracedDim):
        state = state.reshape(tracedDim, -1)
        s = state[:, ind].reshape(-1)
        return np.dot(s,  s.conj())

In [163]:
def tracedProjEfficientv2(state, inds, tracedDim):
        state = state.reshape(tracedDim, -1)
        s = state[:, inds]
        return np.einsum('ji, ji->i', s, s.conj())
        return np.tensordot(s,  s.conj(), axes=[0, 0])

In [164]:
nTrace = 2
out = tracedProjEfficientv2(state, [1, 2], 2**nTrace)
out.shape

(2,)

In [165]:
out

array([8.0899238 +0.j, 4.14486986+0.j])

In [166]:
tracedProjEfficient(state, 1, 2**nTrace)

(8.089923803289576+0j)

In [167]:
tracedProjEfficient(state, 2, 2**nTrace)

(4.1448698590350554+0j)

In [124]:
nQb = 4
partitions = [1, 2, 3]
tracedInd = [[0], [0, 1], [0, 1, 2]]

state = np.random.randn(2**nQb) + 1j*np.random.randn(2**nQb)

In [169]:
for traced, parts in zip(tracedInd, partitions):
    print(parts)
    pTrace = partialTrace(state, traced)
    pTrace = [pTrace[i, i] for i in range(pTrace.shape[0])]
    
    pTraceEfficient = [tracedProjEfficient(state, i, 2**parts) for i in range(2**(nQb - parts))]
    print(np.allclose(pTrace, pTraceEfficient))
    
    pTraceEfficient2 = tracedProjEfficientv2(state, range(2**(nQb - parts)), 2**parts)
    print(np.allclose(pTrace, pTraceEfficient2))
    print()

1
True
True

2
True
True

3
True
True



In [175]:
%%timeit
for traced, parts in zip(tracedInd, partitions):
    pTrace = partialTrace(state, traced)
    # pTrace = [pTrace[i, i] for i in range(pTrace.shape[0])]

54.4 µs ± 949 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [171]:
%%timeit
for traced, parts in zip(tracedInd, partitions):
    pTraceEfficient = [tracedProjEfficient(state, i, 2**parts) for i in range(2**(nQb - parts))]

37.6 µs ± 1.81 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [172]:
%%timeit
for traced, parts in zip(tracedInd, partitions):
    pTraceEfficient2 = tracedProjEfficientv2(state, range(2**(nQb - parts)), 2**parts)

28.1 µs ± 1.94 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [174]:
%%prun
for traced, parts in zip(tracedInd, partitions):
    pTraceEfficient2 = tracedProjEfficientv2(state, range(2**(nQb - parts)), 2**parts)